In [ ]:
pip install pennylane


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 64.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pennylane as qml
import tensorflow as tf
from pathlib import Path
import pickle
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import cv2
# Define constants
PATCH_SIZE = 2  # 2x2 patches
N_QUBITS = 4    # 4 qubits (one per pixel in a patch)
N_QUANTUM_LAYERS = 2  # Number of variational layers

# Create a quantum device
dev = qml.device("default.qubit", wires=N_QUBITS)

# Define the parameterized quantum circuit for feature extraction
@qml.qnode(dev)
def quantum_circuit(patch_data, weights):
    """
    Quantum circuit that encodes patch data and applies parameterized operations

    Args:
        patch_data: Array of 4 pixel values from a 2x2 patch
        weights: Trainable weights for the variational quantum circuit

    Returns:
        Expectation values of different observables
    """
    # Data encoding layer
    for i in range(N_QUBITS):
        # Map pixel value to [0, π]
        qml.RX(patch_data[i] * np.pi, wires=i)

    # Variational quantum layers
    for layer in range(N_QUANTUM_LAYERS):
        # Rotation gates with trainable parameters
        for i in range(N_QUBITS):
            qml.RX(weights[layer, i, 0], wires=i)
            qml.RY(weights[layer, i, 1], wires=i)
            qml.RZ(weights[layer, i, 2], wires=i)

        # Entanglement layer - create a ring of entanglement
        for i in range(N_QUBITS):
            qml.CNOT(wires=[i, (i + 1) % N_QUBITS])

    # Measure different observables for richer feature extraction
    return [
        qml.expval(qml.PauliZ(0)),
        qml.expval(qml.PauliX(1)),
        qml.expval(qml.PauliY(2)),
        qml.expval(qml.PauliZ(3))
    ]

# Initialize random weights for the quantum circuit
def init_quantum_weights():
    # Each qubit has 3 rotation parameters (RX, RY, RZ) for each layer
    return np.random.uniform(
        low=0, high=2*np.pi,
        size=(N_QUANTUM_LAYERS, N_QUBITS, 3)
    )

def quantum_filter_transform(image, weights):
    """
    Transform an image using quantum filters

    Args:
        image: Input image (grayscale, already normalized to [0,1])
        weights: Weights for the quantum circuit

    Returns:
        Quantum features extracted from the image
    """
    h, w = image.shape

    # Calculate output dimensions
    h_out = h // PATCH_SIZE
    w_out = w // PATCH_SIZE

    # Initialize output tensor (4 features per patch)
    quantum_features = np.zeros((h_out, w_out, 4))

    # Process each patch
    for i in range(0, h - PATCH_SIZE + 1, PATCH_SIZE):
        for j in range(0, w - PATCH_SIZE + 1, PATCH_SIZE):
            # Extract patch
            patch = image[i:i+PATCH_SIZE, j:j+PATCH_SIZE]

            # Skip patches that aren't full size
            if patch.shape != (PATCH_SIZE, PATCH_SIZE):
                continue

            # Flatten the patch
            patch_flat = patch.flatten()

            # Apply quantum circuit
            features = quantum_circuit(patch_flat, weights)

            # Store features
            quantum_features[i // PATCH_SIZE, j // PATCH_SIZE, :] = features

    return quantum_features

def save_checkpoint(quantum_features, last_processed_idx, checkpoint_path):
    """Save checkpoint with processed features and last processed index"""
    checkpoint = {
        'quantum_features': quantum_features,
        'last_processed_idx': last_processed_idx
    }
    with open(checkpoint_path, 'wb') as f:
        pickle.dump(checkpoint, f)
    print(f"Checkpoint saved at index {last_processed_idx}")

def load_checkpoint(checkpoint_path):
    """Load checkpoint with processed features and last processed index"""
    with open(checkpoint_path, 'rb') as f:
        checkpoint = pickle.load(f)
    print(f"Checkpoint loaded. Resuming from index {checkpoint['last_processed_idx'] + 1}")
    return checkpoint['quantum_features'], checkpoint['last_processed_idx']

def process_dataset(x_data, quantum_weights, batch_size=10,
                    save_dir="/content/drive/MyDrive/quantum_features",
                    start_idx=0, checkpoint_interval=20):
    """
    Process an entire dataset using quantum circuits with checkpointing

    Args:
        x_data: Array of images [n_samples, height, width, channels]
        quantum_weights: Weights for the quantum circuit
        batch_size: Number of images to process at once (for memory efficiency)
        save_dir: Directory to save the processed features
        start_idx: Index to start processing from (for resuming)
        checkpoint_interval: How often to save checkpoints (number of samples)

    Returns:
        Path to the saved quantum features
    """
    # Create directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Define checkpoint path
    checkpoint_path = os.path.join(save_dir, "processing_checkpoint.pkl")

    # Normalize images to [0,1]
    x_data = x_data.astype(np.float32) / 255.0

    # Get dimensions
    n_samples = x_data.shape[0]
    h, w = x_data.shape[1], x_data.shape[2]
    h_out = h // PATCH_SIZE
    w_out = w // PATCH_SIZE

    # Check if we're resuming from a checkpoint
    if os.path.exists(checkpoint_path) and start_idx == 0:
        quantum_features, last_idx = load_checkpoint(checkpoint_path)
        start_idx = last_idx + 1
    else:
        # Initialize output array
        quantum_features = np.zeros((n_samples, h_out, w_out, 4))
        last_idx = start_idx - 1

    # Process in batches
    for i in range(start_idx, n_samples, batch_size):
        end_idx = min(i + batch_size, n_samples)
        print(f"Processing images {i} to {end_idx-1} of {n_samples}...")

        for j in range(i, end_idx):
            # For grayscale images
            img = x_data[j, :, :, 0]
            quantum_features[j] = quantum_filter_transform(img, quantum_weights)

            # Print progress
            if (j + 1) % 10 == 0:
                print(f"  Processed image {j+1}/{n_samples}")

            # Save checkpoint at intervals
            if (j + 1) % checkpoint_interval == 0 or j == end_idx - 1:
                save_checkpoint(quantum_features, j, checkpoint_path)

    # Save the final quantum features
    features_path = os.path.join(save_dir, "quantum_features.npy")
    np.save(features_path, quantum_features)

    # Save the quantum weights for reproducibility
    weights_path = os.path.join(save_dir, "quantum_weights.npy")
    np.save(weights_path, quantum_weights)

    # Remove the checkpoint file since we've completed processing
    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)

    print(f"Quantum features saved to {features_path}")
    print(f"Quantum weights saved to {weights_path}")

    return features_path

def load_xray_dataset():
    DATASET_DIR = '/content/drive/MyDrive/quantum_thyroid/sorted'
    # Image Size & Quantum Patch Size
    IMG_SIZE = (224, 224)
    classes = ['0','2','3','4a','4b','4c','5']
    images, labels = [],[]
    for label in classes:
        class_dir = os.path.join(DATASET_DIR, str(label))
        for file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, IMG_SIZE) / 255.0
            img = np.stack([img] * 3, axis=-1)
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

def split_and_save_data(x_data, y_data, save_dir="/content/drive/MyDrive/data_splits"):
    """Split data into train/test and save the indices"""
    os.makedirs(save_dir, exist_ok=True)

    # Split the data
    x_train, x_test, y_train, y_test = train_test_split(
        np.arange(len(x_data)), y_data, test_size=0.2, random_state=42, stratify=y_data
    )

    # Save the splits
    splits = {
        'train_indices': x_train,
        'test_indices': x_test,
        'train_labels': y_train,
        'test_labels': y_test
    }

    split_path = os.path.join(save_dir, "data_splits.pkl")
    with open(split_path, 'wb') as f:
        pickle.dump(splits, f)

    print(f"Data splits saved to {split_path}")
    print(f"Train set: {len(x_train)} samples")
    print(f"Test set: {len(x_test)} samples")

    return splits

def load_or_init_weights(weights_path):
    """Load existing weights or initialize new ones"""
    if os.path.exists(weights_path):
        print(f"Loading existing quantum weights from {weights_path}")
        return np.load(weights_path)
    else:
        print("Initializing new quantum weights...")
        return init_quantum_weights()

def main(data_dir="./xray_data", resize=(224, 224), batch_size=10,
         output_dir="/content/drive/MyDrive/quantum_xray_results",
         resume=True, start_idx=0, checkpoint_interval=50):
    """Main function to run the quantum preprocessing pipeline with resume capability"""
    # Make sure the base output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Create subdirectories for features and splits
    quantum_features_dir = os.path.join(output_dir, "quantum_features")
    data_splits_dir = os.path.join(output_dir, "data_splits")
    os.makedirs(quantum_features_dir, exist_ok=True)
    os.makedirs(data_splits_dir, exist_ok=True)

    # Define weights path
    weights_path = os.path.join(quantum_features_dir, "quantum_weights.npy")

    # Check if we need to load the dataset
    checkpoint_path = os.path.join(quantum_features_dir, "processing_checkpoint.pkl")
    processing_needed = True

    if resume and os.path.exists(checkpoint_path):
        print("Checkpoint found. Resuming from previous run.")

        # Load the dataset
        x_data, y_data = load_xray_dataset()

        # Load existing weights or initialize new ones
        quantum_weights = load_or_init_weights(weights_path)

        # Check if data splits exist, create if not
        split_path = os.path.join(data_splits_dir, "data_splits.pkl")
        if os.path.exists(split_path):
            print(f"Loading existing data splits from {split_path}")
            with open(split_path, 'rb') as f:
                splits = pickle.load(f)
        else:
            # Split and save the data indices
            splits = split_and_save_data(x_data, y_data, save_dir=data_splits_dir)
    else:
        # Starting from scratch
        print("Starting new processing run...")

        # Load the dataset
        x_data, y_data = load_xray_dataset()

        # Split and save the data indices
        splits = split_and_save_data(x_data, y_data, save_dir=data_splits_dir)

        # Initialize or load quantum weights
        quantum_weights = load_or_init_weights(weights_path)

    # Process the dataset
    print("Starting quantum feature extraction...")
    features_path = process_dataset(x_data, quantum_weights,
                                   batch_size=batch_size,
                                   save_dir=quantum_features_dir,
                                   start_idx=start_idx,
                                   checkpoint_interval=checkpoint_interval)

    # Save a summary file with information about the processing
    summary_path = os.path.join(output_dir, "processing_summary.txt")
    with open(summary_path, 'w') as f:
        f.write(f"Quantum preprocessing completed on {os.path.basename(__file__)}\n")
        f.write(f"Number of samples processed: {len(x_data)}\n")
        f.write(f"Image resize dimensions: {resize}\n")
        f.write(f"Patch size: {PATCH_SIZE}x{PATCH_SIZE}\n")
        f.write(f"Number of qubits: {N_QUBITS}\n")
        f.write(f"Number of quantum layers: {N_QUANTUM_LAYERS}\n")
        f.write(f"Quantum features saved to: {features_path}\n")
        f.write(f"Data splits saved to: {data_splits_dir}/data_splits.pkl\n")

    print("\nQuantum preprocessing complete!")
    print(f"All results saved to: {output_dir}")
    print(f"You can now run the classification model using the generated quantum")

In [ ]:
if __name__ == "__main__":
    # First, check if Google Drive is mounted
    if not os.path.exists('/content/drive'):
        print("Google Drive is not mounted. Please run the following code first:")
        print("from google.colab import drive")
        print("drive.mount('/content/drive')")
    else:
        main()

Checkpoint found. Resuming from previous run.
Initializing new quantum weights...
Loading existing data splits from /content/drive/MyDrive/quantum_xray_results/data_splits/data_splits.pkl
Starting quantum feature extraction...
Checkpoint loaded. Resuming from index 650
Processing images 650 to 659 of 823...
  Processed image 660/823
Checkpoint saved at index 659
Processing images 660 to 669 of 823...
  Processed image 670/823
Checkpoint saved at index 669
Processing images 670 to 679 of 823...
  Processed image 680/823
Checkpoint saved at index 679
Processing images 680 to 689 of 823...
  Processed image 690/823
Checkpoint saved at index 689
Processing images 690 to 699 of 823...
  Processed image 700/823
Checkpoint saved at index 699
Processing images 700 to 709 of 823...
  Processed image 710/823
Checkpoint saved at index 709
Processing images 710 to 719 of 823...
  Processed image 720/823
Checkpoint saved at index 719
Processing images 720 to 729 of 823...
  Processed image 730/823


NameError: name '__file__' is not defined